<a href="https://colab.research.google.com/github/Siddhant-Thendral-Arasu/Siddhant-Thendral-Arasu/blob/cat_classification/cat_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

cat/not-a-cat classifier using deep neural network, 2 layers, Kaggle dataset, fwd / bkwrd prop, no optimizations, no Keras / Pytorch

In [ ]:
import time
import numpy as np
import h5py
#import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import copy

In [ ]:
def sig(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z

    return A, cache

In [ ]:
def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

In [ ]:
def relu_bkward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [ ]:
def sig_bkward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [ ]:
def data():
    train_dataset = h5py.File('C:\\Users\\t.arasu\\Documents\\Python\\Project1\\dataset_training.h5', "r")
    train_features = np.array(train_dataset["images"][:])
    train_labels = np.array(train_dataset["labels"][:])

    test_dataset = h5py.File('C:\\Users\\t.arasu\\Documents\\Python\\Project1\\dataset_test.h5', "r")
    test_features = np.array(test_dataset["images"][:])
    test_labels = np.array(test_dataset["labels"][:])

    train_labels = train_labels.reshape((1, train_labels.shape[0]))
    test_labels = test_labels.reshape((1, test_labels.shape[0]))

    return train_features, train_labels, test_features, test_labels


In [ ]:
x,y,a,b = data()

In [ ]:
def init_params(n_x, n_h, n_y):

    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1),dtype='float')
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1),dtype='float')

    params = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return params

In [ ]:
def fwd(A_prev, W, b, activation):

    Z, l_cache = np.dot(W, A_prev) + b , (A_prev, W, b)

    if activation == "sig":
        A, a_cache = sig(Z)

    elif activation == "relu":
        A, a_cache = relu(Z)

    cache = (l_cache, a_cache)

    return A, cache

In [ ]:
def cost_(AL, Y):

    m = Y.shape[1]
    cost = -( np.dot(Y, (np.log(AL)).T) + np.dot( (np.ones((Y.shape[0],Y.shape[1]), dtype=int) - Y.astype(int)), (np.log(np.ones((Y.shape[0],Y.shape[1]), dtype=int) - AL)).T))/m
    cost = np.squeeze(cost)

    return cost

In [ ]:
def z_bkprop (dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = np.dot(W.T,dZ)

    return dA_prev, dW, db

In [ ]:
def a_bkprop (dA, cache, activation):
    l_cache, a_cache = cache

    if activation == "relu":
        dZ = relu_bkward(dA,a_cache)

    elif activation == "sig":
        dZ = sig_bkward(dA, a_cache)

    dA_prev, dW, db = z_bkprop(dZ, l_cache)

    return dA_prev, dW, db

In [ ]:
def update_params(params, gradients, learning_rate):
    params = copy.deepcopy(params)
    L = 2

    for l in range(L):
        params["W" + str(l+1)] = params["W" + str(l+1)] - learning_rate * gradients["dW" + str(l + 1)]
        params["b" + str(l+1)] = params["b" + str(l+1)] - learning_rate * gradients["db" + str(l + 1)]

    return params

In [ ]:
train_features, train_labels, test_features, test_labels = data()

In [ ]:
m_train = train_features.shape[0]
num_px = train_features.shape[1]
m_test = test_features.shape[0]
print(train_features.shape)

(8005, 64, 64, 3)


In [ ]:
train_features_flatten = train_features.reshape(train_features.shape[0], -1).T
test_features_flatten = test_features.reshape(test_features.shape[0], -1).T
train_features = train_features_flatten/255.
test_features = test_features_flatten/255.

In [ ]:
def model(X, Y, layers_dims, learning_rate = 0.075, num_iterations = 2500):
    gradients = {}
    costs = []
    m = X.shape[1]
    (n_x, n_h, n_y) = layers_dims

    params = init_params(n_x, n_h, n_y)
    print(len(params))

    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]

    for i in range(0, num_iterations):

        A1, cache1 = fwd(X, W1, b1, "relu")
        A2, cache2 = fwd(A1, W2, b2, "sig")
        cost = cost_(A2, Y)
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        dA1, dW2, db2 = a_bkprop (dA2, cache2, "sig")
        dA0, dW1, db1 = a_bkprop (dA1, cache1, "relu")

        gradients['dW1'] = dW1
        gradients['db1'] = db1
        gradients['dW2'] = dW2
        gradients['db2'] = db2
        params = update_params(params, gradients, learning_rate)

        W1 = params["W1"]
        b1 = params["b1"]
        W2 = params["W2"]
        b2 = params["b2"]

        # Print the cost every 100 iterations
        if i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)

    return params, costs

In [ ]:
print(train_features.shape)
print(train_labels.shape)
print(m_train)
print(num_px)
print(m_test)

(12288, 8005)
(1, 8005)
8005
64
2023


In [ ]:
n_x = 12288
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)
learning_rate = 0.0075

In [ ]:
params, costs = model(train_features, train_labels, layers_dims = (n_x, n_h, n_y))

4
Cost after iteration 0: 0.6931020479889488
Cost after iteration 100: 0.6801787829473863
Cost after iteration 200: 0.6709876704545872
Cost after iteration 300: 0.6631173477796196
Cost after iteration 400: 0.6524377993850805
Cost after iteration 500: 0.6455212992207442
Cost after iteration 600: 0.6246740084958236
Cost after iteration 700: 0.6245096323137374
Cost after iteration 800: 0.6224016131337639
Cost after iteration 900: 0.6471276564746662
Cost after iteration 1000: 0.6306100873170324
Cost after iteration 1100: 0.6277613917744119
Cost after iteration 1200: 0.6875004587903242
Cost after iteration 1300: 0.6116781218870685
Cost after iteration 1400: 0.5894670643565437
Cost after iteration 1500: 0.6007924570974479
Cost after iteration 1600: 0.6016368258214502
Cost after iteration 1700: 0.6311746339167251
Cost after iteration 1800: 0.5958179035170169
Cost after iteration 1900: 0.5822617368389255
Cost after iteration 2000: 0.6463218521364562
Cost after iteration 2100: 0.581981021298345